In [2]:
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile

import numpy as np
import pandas as pd

import os 
from os import getcwd, path, chdir


pwd = getcwd()
templateCase = SolutionDirectory(pwd)
templateCase.name

'/home/timo/Developer/devOpenFOAM/ws_ofUtils/step_valgrind_case_template'

# Objective:

+ Enable design-space-exploration (DSE)

## Parameters
+ Endtime 
+ writeInterval
+ Co numbers
+ Specific changes to BCs
    + myWaveTransmissive: table for the time dependend pressure
+ The domain values for pressure


## Realisation
+ Write one class for each parameter file that performs the changes in the respective file
+ Write one parameter study, performing a DSE with a maximum number of simulations
 

# Figuring out how to access the desired variables in the parsed files

# Control dict

In [3]:
controlDictPath = path.join(templateCase.name, "system", "controlDict")
parsedControlDict = ParsedParameterFile(controlDictPath)
parsedControlDict.content

{'adjustTimeStep': no,
 'application': 'compMultiphaseCavitation',
 'deltaT': 1e-06,
 'endTime': 1.1e-06,
 'functions': {0: '#includeFunc yPlus\n'},
 'maxAcousticCo': 2001,
 'maxAlphaCo': 0.25,
 'maxCo': 0.75,
 'purgeWrite': 0,
 'runTimeModifiable': yes,
 'startFrom': 'latestTime',
 'startTime': 0,
 'stopAt': 'endTime',
 'timeFormat': 'fixed',
 'timePrecision': 6,
 'writeCompression': no,
 'writeControl': 'adjustableRunTime',
 'writeFormat': 'ascii',
 'writeInterval': 1e-06,
 'writePrecision': 9}

In [7]:
df1 = pd.DataFrame(parsedControlDict.content)
df1

,adjustTimeStep,application,deltaT,endTime,functions,maxAcousticCo,maxAlphaCo,maxCo,purgeWrite,runTimeModifiable,startFrom,startTime,stopAt,timeFormat,timePrecision,writeCompression,writeControl,writeFormat,writeInterval,writePrecision
0,no,compMultiphaseCavitation,0.000001,0.000001,#includeFunc yPlus\n,2001,0.25,0.75,0,yes,latestTime,0,endTime,fixed,6,no,adjustableRunTime,ascii,0.000001,9


In [20]:
df1_dict = df1.to_dict('index')
df1_dict = df1_dict.get(0)
print(df1_dict)

{'adjustTimeStep': no, 'application': 'compMultiphaseCavitation', 'deltaT': 1e-06, 'endTime': 1.1e-06, 'functions': '#includeFunc yPlus\n', 'maxAcousticCo': 2001, 'maxAlphaCo': 0.25, 'maxCo': 0.75, 'purgeWrite': 0, 'runTimeModifiable': yes, 'startFrom': 'latestTime', 'startTime': 0, 'stopAt': 'endTime', 'timeFormat': 'fixed', 'timePrecision': 6, 'writeCompression': no, 'writeControl': 'adjustableRunTime', 'writeFormat': 'ascii', 'writeInterval': 1e-06, 'writePrecision': 9}


In [24]:
maxAcousticCo = parsedControlDict["maxAcousticCo"]
maxAlphaCo = parsedControlDict["maxAlphaCo"]
maxCo = parsedControlDict["maxCo"]
endTime = parsedControlDict["endTime"]
writeInterval = parsedControlDict["writeInterval"]

# p field

In [50]:
pFieldPath = path.join(templateCase.name, "0.orig", "p")
parsedPFieldPath = ParsedParameterFile(pFieldPath)
parsedPFieldPath.content

{'boundaryField': {'frontAndBack': {'type': 'empty'},
  'inlet': {'type': 'calculated', 'value': '$internalField'},
  'lowerWall': {'type': 'calculated', 'value': '$internalField'},
  'outlet': {'type': 'calculated', 'value': '$internalField'},
  'upperWall': {'type': 'calculated', 'value': '$internalField'}},
 'dimensions': '[ 1 -1 -2 0 0 0 0 ]',
 'internalField': 'uniform 101001'}

In [52]:
parsedPFieldPath["boundaryField"]["inlet"]["type"]

'calculated'

In [46]:
df2 = pd.DataFrame(parsedPFieldPath.content.get("boundaryField"))
df2["internalField"] = parsedPFieldPath.content.get("internalField")
df2["dimensions"] = parsedPFieldPath.content.get("dimensions")

df2

,frontAndBack,inlet,lowerWall,outlet,upperWall,internalField,dimensions
type,empty,calculated,calculated,calculated,calculated,uniform 101001,[ 1 -1 -2 0 0 0 0 ]
value,NaN,$internalField,$internalField,$internalField,$internalField,uniform 101001,[ 1 -1 -2 0 0 0 0 ]


In [30]:
df2.loc["frontAndBack"]["boundaryField"]

{'type': 'empty'}

In [45]:
p_internalField = parsedPFieldPath["internalField"]

p_internalField

'uniform 101000'

# p_rgh field

In [53]:
p_rghFieldPath = path.join(templateCase.name, "0.orig", "p_rgh")
parsedP_rghFieldPath = ParsedParameterFile(p_rghFieldPath)
parsedP_rghFieldPath.content

{'boundaryField': {'frontAndBack': {'type': 'empty'},
  'inlet': {'type': 'uniformFixedValue',
   'uniformValue': ['table', [[0, 101000], [0.075, 101000], [0.085, 14000]]]},
  'lowerWall': {'type': 'zeroGradient'},
  'outlet': {'type': 'fixedValue', 'value': 'uniform 14000'},
  'upperWall': {'type': 'zeroGradient'}},
 'dimensions': '[ 1 -1 -2 0 0 0 0 ]',
 'internalField': 'uniform 101000'}

In [57]:
parsedP_rghFieldPath.content["boundaryField"]["inlet"]["uniformValue"]

['table', [[0, 101000], [0.075, 101000], [0.085, 14000]]]

In [118]:
df4 = pd.DataFrame(parsedP_rghFieldPath.content["boundaryField"]["inlet"]["uniformValue"][1], columns=["t", "p"])
df4.index = ["inlet","inlet","inlet"]
df4

,t,p
inlet,0.000,101000
inlet,0.075,101000
inlet,0.085,14000


In [114]:
df4.iloc[1:,0] = df4.iloc[1:]["t"] + 0.1

df4

,t,p
inlet,0.000,101000
inlet,0.175,101000
inlet,0.185,14000


In [116]:
df3 = pd.DataFrame(parsedP_rghFieldPath.content.get("boundaryField"))
df3["internalField"] = parsedP_rghFieldPath.content.get("internalField")
df3["dimensions"] = parsedP_rghFieldPath.content.get("dimensions")
dft = df4

df3 = pd.concat([df3, dft])


df3

,dimensions,frontAndBack,inlet,internalField,lowerWall,outlet,p,t,upperWall
type,[ 1 -1 -2 0 0 0 0 ],empty,uniformFixedValue,uniform 101000,zeroGradient,fixedValue,NaN,NaN,zeroGradient
uniformValue,[ 1 -1 -2 0 0 0 0 ],NaN,"[table, [[0, 101000], [0.075, 101000], [0.085,...",uniform 101000,NaN,NaN,NaN,NaN,NaN
value,[ 1 -1 -2 0 0 0 0 ],NaN,NaN,uniform 101000,NaN,uniform 14000,NaN,NaN,NaN
inlet,NaN,NaN,NaN,NaN,NaN,NaN,101000.0,0.000,NaN
inlet,NaN,NaN,NaN,NaN,NaN,NaN,101000.0,0.175,NaN
inlet,NaN,NaN,NaN,NaN,NaN,NaN,14000.0,0.185,NaN


In [ ]:
dft_list = 